In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import pickle
import json
import gc
from copy import copy, deepcopy
from collections import defaultdict, Counter
import requests
import xmltodict

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import log_loss
from scipy import sparse
from scipy.special import expit as sigmoid
import scipy.stats as sts
from datetime import timedelta, datetime
import warnings

warnings.filterwarnings("ignore")

## 1. Считаем данные


In [2]:
%%time

#Считаем данные, разобьем на трейн и валдацию и сохраним в папку

with open('data/results.pkl', 'rb') as file:
    results = pickle.load(file)

with open('data/tournaments.pkl', 'rb') as file:
    tournaments = pickle.load(file)


date_train = datetime.strptime('2019-01-01', '%Y-%m-%d')
date_val = datetime.strptime('2020-01-01', '%Y-%m-%d')


max_player_id = 0
max_team_id = 0

tournaments_train = []
tournaments_val = []

for key in tqdm(tournaments.keys(), position=0, leave=False):
    tournament_date = datetime.fromisoformat(
        tournaments[key]['dateStart']).replace(tzinfo=None)
    if tournament_date >= date_train:

        if not results[key]:
            continue

        total_questions = set()
        for team in results[key]:
            if team.get('mask') is not None:
                total_questions.add(len(team['mask']))

        if len(total_questions) > 1:
            continue

        tournament = dict()
        tournament['id'] = tournaments[key]['id']
        tournament['teams'] = []

        for team in results[key]:
            # Уберем команды, где нет ответов или в ответах есть 'X', '?'
            if team.get('mask') is None or team.get('mask').replace('1', '').replace('0', ''):
                continue

            if team['team']['id'] > max_team_id:
                max_team_id = team['team']['id']

            team_dict = dict()
            team_dict['id'] = team['team']['id']
            team_dict['mask'] = team['mask']
            team_dict['members'] = []

            for member in team['teamMembers']:

                player_id = member['player']['id']
                if player_id > max_player_id:
                    max_player_id = player_id

                team_dict['members'].append(player_id)

            tournament['teams'].append(team_dict)

        if not tournament['teams']:
            continue

        if tournament_date < date_val:
            tournaments_train.append(tournament)
        else:
            tournaments_val.append(tournament)

with open('data/tournaments_train.json', 'w') as file:
    json.dump(tournaments_train, file)

with open('data/tournaments_val.json', 'w') as file:
    json.dump(tournaments_val, file)

del results, tournaments

gc.collect()



with open('data/tournaments_train.json', 'r') as file:
    tournaments_train = json.load(file)

with open('data/tournaments_val.json', 'r') as file:
    tournaments_val = json.load(file)

with open('data/players.pkl', 'rb') as file:
    players = pickle.load(file)

Wall time: 22.9 s


## 2. Baseline



In [3]:
# Построим словари по id игрока -> индекс и обратный.

members = set()
member_answers = defaultdict(int)
member_right_answers = defaultdict(int)
member_tours = defaultdict(int)

for tournament in tournaments_train:
    for team in tournament['teams']:
        for member in team['members']:
            member_answers[member] += len(team['mask'])
            member_right_answers[member] = sum(list(map(int, team['mask'])))
            member_tours[member] += 1
            members.add(member)
        

member_to_idx = {member:idx for idx, member in enumerate(members)}
idx_to_members = {idx:member for member, idx in member_to_idx.items()}

print(f'Всего игроков в обучающей выборке: {len(member_to_idx)}')

Всего игроков в обучающей выборке: 55151


In [4]:
%%time

# Соберем обучающую выборку.

# - Объект - 2 сконкатенированных onehot-вектора: вектор всех игроков + вектор всех вопросов.
# - Таргет - ответил или нет игрок на вопрос.

member_idxs = []
question_idxs = []
team_ids = []
tournament_ids = []
results = []
member_questions_count = []

questions_count = 0
for tournament in tournaments_train:
    tour_questions_count = len(tournament['teams'][0]['mask'])
    for team in tournament['teams']:
        team_answers = list(map(int, team['mask']))
        for q in range(tour_questions_count):
            for member in team['members']:
                member_idxs.append(member_to_idx[member])
                question_idxs.append(len(member_to_idx) + questions_count + q)
                team_ids.append(team['id'])
                tournament_ids.append(tournament['id'])
                results.append(team_answers[q])
                member_questions_count.append(member_answers[member])
    questions_count += tour_questions_count
    
    
X = sparse.lil_matrix((len(member_idxs), len(member_to_idx) + questions_count),  dtype=int)
X[range(len(member_idxs)), member_idxs] = 1
X[range(len(member_idxs)), question_idxs] = 1
y = np.array(results)

dim0, dim1 = X.shape 


Wall time: 42.8 s


In [5]:
%%time

# В качестве рейтинга можем взять коэфициенты логистической регрессии. 
# Достанем рейтинг игроков и рейтинг вопросов.

lr = LogisticRegression(solver='saga')
lr.fit(X, y)


question_rating = lr.coef_[0][len(member_to_idx):]
rating = lr.coef_[0][:len(member_to_idx)]
rating_list = []
for idx, member in idx_to_members.items():
    item = {
        'score': rating[idx],
        'id': member,
        'name': players[member]['name'] + ' ' + players[member]['surname'],
        'questions_count': member_answers[member],
    }
    rating_list.append(item)
    
sorted_rating = sorted(rating_list, key=lambda x: x['score'], reverse=True)

Wall time: 4min 41s


In [6]:
def get_member_position(id):
    url = f'https://rating.chgk.info/api/players/{id}/rating/last'
    position = -1
    try:
        position = requests.get(url).json()['rating_position']
        position = int(position)
    except Exception as e:
        pass
    
    return position

In [7]:
%%time
# получилось не прям идеально, но сойдет)

df_rating = pd.DataFrame(sorted_rating)[:50]
df_rating['actual_position'] = df_rating['id'].apply(get_member_position)
df_rating['questions_count'] = df_rating['id'].apply(lambda x: member_answers[x])
df_rating.head(50)

Wall time: 8.63 s


,score,id,name,questions_count,actual_position
0,4.069293,27403,Максим Руссо,1796,5
1,3.932572,4270,Александра Брутер,2240,6
2,3.885220,28751,Иван Семушин,3071,3
3,3.853065,27822,Михаил Савченков,2666,2
4,3.758544,30270,Сергей Спешков,3017,4
5,3.752905,30152,Артём Сорожкин,4006,1
6,3.619237,18036,Михаил Левандовский,1113,8
7,3.614460,20691,Станислав Мереминский,1370,38
8,3.542883,87637,Антон Саксонов,927,158
9,3.539573,22799,Сергей Николенко,1806,10


In [8]:
%%time

#проверим сколько

top_50 = df_rating[df_rating['actual_position'] <= 50].shape[0]

print(f'В топ-50 рейтинга модели попали {top_50} игроков из топ-50 реального рейтинга')

В топ-50 рейтинга модели попали 26 игроков из топ-50 реального рейтинга
Wall time: 2 ms


## 3. Оценка качества


In [9]:

# Уберем из тестовой выборки новых участников, о которых у нас нет данных в обучающей выборке. 
# Заодно, сразу и посчитаем число правильных ответов в командах.


tournaments_test = []
tournaments_rating_true = []

for tournament in tournaments_val:
    teams_test = []
    teams_rating = []
    for team in tournament['teams']:
  
        memebers = [member for member in team['members'] if member in member_to_idx.keys()]
        team_test = copy(team)
        team_test['members'] = memebers
        
        if len(memebers) > 0:
            teams_test.append(team_test)
            team_answers = list(map(int, team['mask']))
            team_score = sum(team_answers)
            teams_rating.append(team_score)
     
    if len(teams_test) > 1:
        tournament_test = copy(tournament)
        tournament_test['teams'] = teams_test
        tournaments_test.append(tournament_test)
        tournaments_rating_true.append(teams_rating)


В качестве рейтинга команды будем использовать вероятность того, что хотя бы 1 игрок команды ответил на вопрос правильно.

Т.к. вопросы у нас неизвестны и важны не абсолютные значения числа отвеченных вопросов, то наша задача - сравнить "силу" команд. Для этого возьмем 1 вопрос. Т.к. коэфициенты, которые мы обучили для вопросов, распределены вокруг нуля, возьмем вопрос с коэффициентом 0. Для этого нам вообще ничего не нужно делать.

In [10]:
def correlations(tournaments_rating_true, tournaments_rating_pred):

    spearmanr_corrs = []
    kendall_corrs = []
    for i in range(len(tournaments_rating_true)):
        spearman = sts.spearmanr(
            tournaments_rating_true[i], tournaments_rating_pred[i]).correlation
        kendall = sts.kendalltau(
            tournaments_rating_true[i], tournaments_rating_pred[i]).correlation
        spearmanr_corrs.append(spearman)
        kendall_corrs.append(kendall)

    print(f'Корреляция Спирмена: {np.mean(spearmanr_corrs):.4f}')
    print(f'Корреляция Кендалла: {np.mean(kendall_corrs):.4f}')


def predict_tournaments(model, tournaments, member_to_idx):

    tournaments_rating_pred = []
    for torunament in tqdm(tournaments, position=0, leave=False):
        tournament_questions_count = len(tournament['teams'][0]['mask'])
        preds = []
        for team in torunament['teams']:
            memeber_idxs = [member_to_idx[member]
                            for member in team['members']]
            members_count = len(memeber_idxs)

            X = sparse.lil_matrix((members_count, dim1), dtype=int)
            X[range(len(memeber_idxs)), memeber_idxs] = 1

            fail_probas = model.predict_proba(X)[:, 0]
            team_proba = 1 - fail_probas.prod()
            preds.append(team_proba)

        tournaments_rating_pred.append(preds)
    return tournaments_rating_pred

In [11]:
%%time 

tournaments_rating_pred = predict_tournaments(
    model=lr, 
    tournaments=tournaments_test,
    member_to_idx=member_to_idx, 
)

correlations(tournaments_rating_true, tournaments_rating_pred)

Корреляция Спирмена: 0.8025
Корреляция Кендалла: 0.6447
Wall time: 3.46 s


## 4. EM-алгоритм


В качестве вектора скрытых переменных будем использовать вероятность ответа игроком на вопрос при условии команды: $z = P(member=1|team)$

Сделаем предположение, что если команда не ответила на вопрос, то никто в команде не ответил на вопрос (на самом деле, ответ мог прозвучать, но его не приняли участники). Т.е. $P(member=1|team=0) = 0$

- E-шаг: предсказываем вероятности ответа на вопрос игрока при условии команды: 
$P(member=1|team=1) = \frac{P(member=1 \cap team=1)}{P(team=1)} = \frac{P(team=1 | member=1) P(member=1)}{P(team=1)} = \frac{P(member=1)}{P(team=1)}$
- M-шаг: максимизируем правдоподобие. Обучаем модель на вероятностях с E-шага

In [12]:
%%time 
#За начальное приближение возьмем z возьмем ответы каждого игрока команды, как будте он играет независимо (как в baseline). 
# Обучим логистическую регрессиию, будем использовать ее веса как точку старта. Это просто быстрее.

lr_initial = LogisticRegression(tol=1e-1, solver='saga', penalty='none')
lr_initial.fit(X, y)

Wall time: 1min 1s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=None, solver='saga', tol=0.1, verbose=0,
                   warm_start=False)

In [13]:
#Здесь понадобится немного модифицированная логистическая регрессия. 
# Обычная из sklearn принимает метки класса, а нужно, чтобы логистическая регрессия принимала вероятности на вход.
# Будем обучать стохастическим градиентным спуском по батчам.

class ProbabilityEstimator:
    def __init__(self, init_classifier=None):
        if init_classifier:
            self.w = np.hstack((init_classifier.intercept_, init_classifier.coef_[0]))
        else:
            self.w = None
        
    def fit(self, X, y, max_iters=100000, lr=10, batch_size=1000, tol=0.0000001):
        X = sparse.hstack([np.ones(len(y)).reshape(-1, 1), X], format='csr')
        if self.w is None:
            self.w = np.random.normal(size=X.shape[1])
            
        losses = []
        prev_mean_loss = np.inf
        
        data_len = X.shape[0]
        
        for i in range(max_iters):
            batch_idxs = np.random.choice(data_len, batch_size)
            X_batch = X[batch_idxs, :]
            y_batch = y[batch_idxs]
            preds = sigmoid(X_batch.dot(self.w))
            losses.append(self.log_loss(y_batch, preds))
            if (i + 1) % 1000 == 0:
                new_mean_loss = np.mean(losses)
                if (prev_mean_loss - new_mean_loss) < tol:
                    print(f'Fitted. Mean loss: {new_mean_loss:.4f}', flush=True)
                    break                    
                losses = []
                prev_mean_loss = new_mean_loss

            grad = -X_batch.T.dot(y_batch - preds) / len(y_batch)
            self.w -= lr * grad 
    
    def predict_proba(self, X):
        X = sparse.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
        preds = sigmoid(X.dot(self.w))
        return np.hstack(((1 - preds).reshape(-1, 1), preds.reshape(-1, 1)))
    
    @staticmethod
    def log_loss(y, p):
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

In [14]:
%%time 


estimator = ProbabilityEstimator(init_classifier=lr_initial)

tournaments_rating_pred = predict_tournaments(
    model=estimator, 
    tournaments=tournaments_test,
    member_to_idx=member_to_idx, 
)
correlations(tournaments_rating_true, tournaments_rating_pred)

for _ in range(5):
    
    # E-шаг
    preds = estimator.predict_proba(X)
    
    # Вероятность ответа игрока при условии команды
    df_team = pd.DataFrame({'team': team_ids, 'question': question_idxs, 'fail_pred': preds[:, 0], 'success_pred': preds[:, 1]})
    df_team_pis = df_team.groupby(['team', 'question']).agg({'fail_pred': 'prod'}).reset_index()
    df_team_pis['team_success_pred'] = 1 - df_team_pis['fail_pred']
    df_team_pis.drop(columns=['fail_pred'], inplace=True)
    df_team = pd.merge(df_team, df_team_pis, left_on=['team', 'question'],  right_on=['team', 'question'])
    z = (df_team['success_pred'] / df_team['team_success_pred']).clip(0, 1)
    
    # По нашему предположению, если команда не ответила на вопрос, 
    # то никто из команды не ответил
    z[y == 0] = 0
    
    
    # M-шаг
    estimator.fit(X, z, lr=20)

    tournaments_rating_pred = predict_tournaments(
        model=estimator, 
        tournaments=tournaments_test,
        member_to_idx=member_to_idx, 
    )
    correlations(tournaments_rating_true, tournaments_rating_pred)
    




Корреляция Спирмена: 0.8070
Корреляция Кендалла: 0.6497
Fitted. Mean loss: 0.5000


Корреляция Спирмена: 0.8157
Корреляция Кендалла: 0.6602
Fitted. Mean loss: 0.2962


Корреляция Спирмена: 0.8200
Корреляция Кендалла: 0.6644
Fitted. Mean loss: 0.3043


Корреляция Спирмена: 0.8228
Корреляция Кендалла: 0.6674
Fitted. Mean loss: 0.3104


Корреляция Спирмена: 0.8229
Корреляция Кендалла: 0.6681
Fitted. Mean loss: 0.3132


Корреляция Спирмена: 0.8231
Корреляция Кендалла: 0.6683
Wall time: 21min 13s


In [15]:
%%time 

#Метрики действительно выросли. Посмотрим на новый рейтинг игроков.


question_rating = estimator.w[len(member_to_idx) + 1:]
rating = estimator.w[1:len(member_to_idx) + 1]
rating_list = []
for idx, member in idx_to_members.items():
    item = {
        'score': rating[idx],
        'id': member,
        'name': players[member]['name'] + ' ' + players[member]['surname'],
        'questions_count': member_answers[member],
    }
    rating_list.append(item)
    
sorted_rating = sorted(rating_list, key=lambda x: x['score'], reverse=True)

Wall time: 104 ms


In [16]:
%%time 


df_rating_em = pd.DataFrame(sorted_rating)[:50]
df_rating_em['actual_position'] = df_rating_em['id'].apply(get_member_position)
df_rating_em['questions_count'] = df_rating_em['id'].apply(lambda x: member_answers[x])
df_rating_em.head(50)

Wall time: 8.42 s


,score,id,name,questions_count,actual_position
0,4.793476,74001,Игорь Мокин,900,74
1,4.598408,27403,Максим Руссо,1796,5
2,4.562915,20691,Станислав Мереминский,1370,38
3,4.560792,22474,Илья Немец,75,4471
4,4.528266,4270,Александра Брутер,2240,6
5,4.492654,21698,Александр Мосягин,838,146
6,4.477140,27822,Михаил Савченков,2666,2
7,4.469230,30152,Артём Сорожкин,4006,1
8,4.463901,30260,Евгений Спектор,233,1584
9,4.432340,22935,Илья Новиков,1266,132


In [17]:
%%time 

top_50 = df_rating_em[df_rating_em['actual_position'] <= 50].shape[0]

print(f'В топ-50 рейтинга модели попали {top_50} игроков из топ-50 реального рейтинга')

В топ-50 рейтинга модели попали 22 игроков из топ-50 реального рейтинга
Wall time: 3 ms


In [18]:
%%time


THRESHOLD = 1000
print(
    f"В baseline модели в топ-50 было {(df_rating['questions_count'] > THRESHOLD).sum()} игроков, сыгравших меньше {THRESHOLD} вопросов, в EM-модели - {(df_rating_em['questions_count'] < THRESHOLD).sum()}")

В baseline модели в топ-50 было 34 игроков, сыгравших меньше 1000 вопросов, в EM-модели - 24
Wall time: 2 ms


Игроков которые ответили меньше чем на 1000 вопросовj стал меньше

## 5. Рейтинг турниров по сложности вопросов



In [19]:
%%time


# Посчитаем среднюю сложность вопросов в турнире. Для этого возьмем коэффициенты нашей модели, связанные с вопросами.
# Самый сложный вопрос - самый маленький коэффициент.

with open('data/tournaments.pkl', 'rb') as file:
    tournaments = pickle.load(file)

tournament_rating = dict()
question_rating = estimator.w[1 + len(member_to_idx):]
question_to_tour = dict()

questions_count = 0
for tournament in tournaments_train:
    tour_questions_count = len(tournament['teams'][0]['mask'])
    start_index, end_index = questions_count, questions_count + tour_questions_count
    tournament_rating[tournament['id']] = np.mean(
        question_rating[start_index: end_index])

    for q in range(tour_questions_count):
        question_to_tour[questions_count + q] = {
            'tournament_id': tournament['id'],
            'tournament_name': tournaments[tournament['id']]['name'],
            'question_num': q + 1,
        }
    questions_count += tour_questions_count

tournament_rating = sorted(tournament_rating.items(), key=lambda x: x[1])
df_tournament_rating = pd.DataFrame(tournament_rating, columns=[
                                    'id', 'rating']).drop(columns=['rating'])
df_tournament_rating['name'] = df_tournament_rating['id'].apply(
    lambda x: tournaments[x]['name'])

Wall time: 42 ms


In [20]:
print('Топ-10 турниров с самыми сложными вопросами')
df_tournament_rating.head(10)

Топ-10 турниров с самыми сложными вопросами


,id,name
0,6149,Чемпионат Санкт-Петербурга. Первая лига
1,5928,Угрюмый Ёрш
2,6101,Воображаемый музей
3,5159,Первенство правого полушария
4,5930,Зеркало мемориала памяти Михаила Басса
5,5943,Чемпионат Мира. Этап 2 Группа С
6,5587,Записки охотника
7,5942,Чемпионат Мира. Этап 2. Группа В
8,5465,Чемпионат России
9,5427,День D


In [21]:
print('Топ-10 турниров с самыми легкими вопросами')
df_tournament_rating.tail(10)

Топ-10 турниров с самыми легкими вопросами


,id,name
594,5701,Школьный Синхрон-lite. Выпуск 3.3
595,5702,(а)Синхрон-lite. Лига старта. Эпизод IX
596,5697,Школьный Синхрон-lite. Выпуск 3.1
597,5698,(а)Синхрон-lite. Лига старта. Эпизод VII
598,5457,Студенческий чемпионат Калининградской области
599,5954,Школьная лига. II тур.
600,5012,Школьный Синхрон-lite. Выпуск 2.5
601,5013,(а)Синхрон-lite. Лига старта. Эпизод V
602,5955,Школьная лига. III тур.
603,5936,Школьная лига. I тур.


Как видим, по названиям, многие турниры с легкими вопросами имею приписку школа, или старт, а турниры со слодными вопросами уже имеют приписку городов или мира